# Caipi
Learning toghether system based on explanatory interactive learning.

[Original paper](https://dl.acm.org/doi/10.1145/3306618.3314293) |
[Implementation](https://github.com/msetzu/hdms-essai24) |
[Original implementation](https://github.com/stefanoteso/caipi)


## Basic idea

1. Train a model **(machine step)**
2. Query the model for uncertainty: what are the weakest predictions?
3. Construct explanations for said instances
4. Present explanation to user as an artifact
5. User corrects the artifact **(human step)**
6. Generate auxiliary data on the basis of the artifact
7. Finetune the model **(machine step)**

## Setup

In [ ]:
!pip install -r requirements.caipidemo.txt

In [2]:
import copy
import pprint

from datasets import load_dataset
from sklearn.metrics import classification_report

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from coipee import Coipee

/home/ms/.virtualenvs/essai_caipi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load data

In [3]:
########
# Data #
########
dataset = load_dataset("mstz/adult", "income")["train"].to_pandas()
dataset = dataset.select_dtypes(include="number")
data = dataset.values
features, labels = data[:, :-1], data[:, -1]

features_train, features_test, labels_train, labels_test = train_test_split(features, labels,
                                                                            stratify=labels, random_state=1)

### Train a toy model

In [4]:
########
# Model #
########
def fit_model(model, x, y):
    model.fit(x, y)

    return model

print("Training model...")
base_model = MLPClassifier(random_state=1, max_iter=300)
base_model = fit_model(base_model, features_train, labels_train)
predicted_labels_test = base_model.predict(features_test)
base_report = classification_report(labels_test, predicted_labels_test)
pprint.pp(base_report)

Training model...
('              precision    recall  f1-score   support\n'
 '\n'
 '         0.0       0.81      0.94      0.87      6967\n'
 '         1.0       0.61      0.31      0.41      2191\n'
 '\n'
 '    accuracy                           0.79      9158\n'
 '   macro avg       0.71      0.62      0.64      9158\n'
 'weighted avg       0.76      0.79      0.76      9158\n')


# Caipi

Now we can create our instance:

In [5]:
barman = Coipee(
    model=base_model,
    fit_model=fit_model,
    pool=features_train,
    pool_labels=labels_train,
    names=dataset.columns.tolist()
)

### ...and query for uncertain instances

In [6]:
# Machine step: retrieve errors
print("Querying...")
artifact = barman.query(number_of_instances=100)
print(f"Explanation: {artifact}")

Querying...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.78it/s]

Explanation: Number of explained instances: 100
Explanation:
{'age': True,
 'capital_gain': True,
 'capital_loss': True,
 'education': True,
 'final_weight': True,
 'hours_worked_per_week': True}


The explanation is a feature mask: features important to the model are marked as `True`, while others as `False`.

We can also threshold importance at different levels: the higher the threshold, the higher the required importance
to mark a feature as important:

In [7]:
artifact = barman.query(10, threshold=0.01)
print(artifact.explanation)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.69it/s]

[ True  True  True  True  True  True]


Once we have our explanation, we can correct it by marking some important features as not important, and vice versa:

In [8]:
corrected_artifact = copy.deepcopy(artifact)

corrected_artifact.explanation[:] = False
corrected_artifact.explanation[[0, 1, 2]] = True

Here, we have simply said to the model that actually, only the features `0, 1, 2` are actually important.
We can also directly retrieve differences between artifacts through the `diff` method:

In [9]:
print(f"Difference: {artifact.diff(corrected_artifact)}")

Difference: {('education', 3), ('final_weight', 4), ('hours_worked_per_week', 5)}


Now that we have corrected the explanation, we can feed it back to the model:

In [10]:
barman.stack_correction(corrected_artifact)  # adds the correction to correction stack of the model
barman.correct_model()  # triggers a training phase

In [11]:
barman.model

MLPClassifier(max_iter=300, random_state=1)